<a href="https://colab.research.google.com/github/bmybest/ML-for-Non-stationary-Data-in-Financial-Market/blob/main/Final_verion_tranfer_learning_with_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objs as go
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
from pandas.core.frame import validate_axis_style_args
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_absolute_percentage_error
import math

In [ ]:
df = pd.read_csv('SET.5.20.csv')

X = df[['mkt', 'smb', 'hml', 'rmw', 'cma', 'umd']]
y = df["return 20d"]

X_train,X_test = X[0:600],X[609:620]
y_train, y_test= y[0:600],y[609:620]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n=692
ns=2 #number set.seed

y_pred_tf=[]

MAE=[]
MSE=[]
RMSE=[]

MAPE=[]

In [ ]:
np.random.seed(ns)
tf.random.set_seed(ns)
keras.utils.set_random_seed(ns)
model_A = keras.models.Sequential([
    keras.layers.Dense(5, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(4, activation="relu"),
    keras.layers.Dense(4, activation="relu"),
    keras.layers.Dense(4, activation="relu"),
    keras.layers.Dense(4, activation="relu"),
    keras.layers.Dense(1)
])
model_A.compile(loss="mean_absolute_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model_A.fit(X_train, y_train, epochs=150)
mse_test = model_A.evaluate(X_test, y_test)
#X_new = X_test[:3]
y_pred = model_A.predict(X_test)
y_pred_tf.append(y_pred)

Epoch 1/150
19/19 [==============================] - 2s 4ms/step - loss: 0.0706
Epoch 2/150
19/19 [==============================] - 0s 5ms/step - loss: 0.0637
Epoch 3/150
19/19 [==============================] - 0s 4ms/step - loss: 0.0596
Epoch 4/150
19/19 [==============================] - 0s 3ms/step - loss: 0.0574
Epoch 5/150
19/19 [==============================] - 0s 5ms/step - loss: 0.0558
Epoch 6/150
19/19 [==============================] - 0s 4ms/step - loss: 0.0543
Epoch 7/150
19/19 [==============================] - 0s 3ms/step - loss: 0.0531
Epoch 8/150
19/19 [==============================] - 0s 4ms/step - loss: 0.0521
Epoch 9/150
19/19 [==============================] - 0s 4ms/step - loss: 0.0511
Epoch 10/150
19/19 [==============================] - 0s 3ms/step - loss: 0.0502
Epoch 11/150
19/19 [==============================] - 0s 4ms/step - loss: 0.0494
Epoch 12/150
19/19 [==============================] - 0s 8ms/step - loss: 0.0487
Epoch 13/150
19/19 [=================

In [ ]:
h1 = go.Scatter(y=history.history['loss'],mode="lines",
line=dict(width=2,color='blue'),name="loss")

data = [h1]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='Epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1, filename='Underfit')

In [ ]:
y_test

609   -0.010543
610   -0.007841
611   -0.020905
612   -0.031525
613   -0.025447
614   -0.029129
615   -0.027935
616   -0.032069
617   -0.010115
618    0.003218
619   -0.010454
Name: return 20d, dtype: float64

In [ ]:
y_pred

array([[-0.00935416],
       [-0.00935416],
       [-0.00935416],
       [ 0.02247113],
       [-0.00935416],
       [-0.00935416],
       [-0.00935416],
       [-0.00935416],
       [-0.00935416],
       [-0.00935416],
       [-0.00935416]], dtype=float32)

In [ ]:
MAE = metrics.mean_absolute_error(y_test,y_pred)
MSE = metrics.mean_squared_error(y_test,y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
MAPE=mean_absolute_percentage_error(y_test, y_pred)*100
print("MAE = ",MAE)
print("MSE = ",MSE)
print("RMSE = ",RMSE)
print("MAPE = ",MAPE)

MAE =  0.01453133191863476
MSE =  0.0004294363007740879
RMSE =  0.02072284490059432
MAPE =  84.93515679895079


In [ ]:
for i in range(n-1):
  model_A_clone = keras.models.clone_model(model_A)
  model_A_clone.set_weights(model_A.get_weights())
  model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
  model_B_on_A.add(keras.layers.Dense(1))

  for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

  model_B_on_A.compile(loss="mean_absolute_error",
                       optimizer=keras.optimizers.SGD(learning_rate=1e-3))

  df = pd.read_csv('SET.5.20.csv')

  X = df[['mkt', 'smb', 'hml', 'rmw', 'cma', 'umd']]
  y = df["return 20d"]

  X_train_B,X_test_B = X[i+1:i+601],X[i+620:i+621]
  y_train_B, y_test_B= y[i+1:i+601],y[i+620:i+621]

  scaler = StandardScaler()
  X_train_B = scaler.fit_transform(X_train_B)
  X_test_B = scaler.transform(X_test_B)

  np.random.seed(ns)
  tf.random.set_seed(ns)
  keras.utils.set_random_seed(ns)
  history_B = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,)

  for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

  model_B_on_A.compile(loss="mean_absolute_error",
                       optimizer=keras.optimizers.SGD(learning_rate=1e-3))
  history_B = model_B_on_A.fit(X_train_B, y_train_B, epochs=70)
  mse_test_B = model_B_on_A.evaluate(X_test_B, y_test_B)
  y_pred_B = model_B_on_A.predict(X_test_B)
  y_pred_tf.append(y_pred_B)

  model_B_on_A = model_A

Epoch 1/4
19/19 [==============================] - 1s 3ms/step - loss: 0.0415
Epoch 2/4
19/19 [==============================] - 0s 4ms/step - loss: 0.0411
Epoch 3/4
19/19 [==============================] - 0s 7ms/step - loss: 0.0410
Epoch 4/4
19/19 [==============================] - 0s 2ms/step - loss: 0.0409
Epoch 1/70
19/19 [==============================] - 1s 9ms/step - loss: 0.0408
Epoch 2/70
19/19 [==============================] - 0s 11ms/step - loss: 0.0407
Epoch 3/70
19/19 [==============================] - 0s 4ms/step - loss: 0.0407
Epoch 4/70
19/19 [==============================] - 0s 5ms/step - loss: 0.0406
Epoch 5/70
19/19 [==============================] - 0s 3ms/step - loss: 0.0406
Epoch 6/70
19/19 [==============================] - 0s 4ms/step - loss: 0.0406
Epoch 7/70
19/19 [==============================] - 0s 3ms/step - loss: 0.0405
Epoch 8/70
19/19 [==============================] - 0s 5ms/step - loss: 0.0405
Epoch 9/70
19/19 [==============================] - 0s 

1/1 [==============================] - 0s 181ms/step - loss: 0.0372


1/1 [==============================] - 0s 103ms/step
Epoch 1/4
19/19 [==============================] - 1s 3ms/step - loss: 0.0424
Epoch 2/4
19/19 [==============================] - 0s 3ms/step - loss: 0.0422
Epoch 3/4
19/19 [==============================] - 0s 3ms/step - loss: 0.0421
Epoch 4/4
19/19 [==============================] - 0s 2ms/step - loss: 0.0420
Epoch 1/70
19/19 [==============================] - 1s 3ms/step - loss: 0.0420
Epoch 2/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0418
Epoch 3/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0416
Epoch 4/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0415
Epoch 5/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0414
Epoch 6/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0413
Epoch 7/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0412
Epoch 8/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0411
Epo

1/1 [==============================] - 0s 113ms/step - loss: 0.0484


Streaming output truncated to the last 5000 lines.
Epoch 47/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0287
Epoch 48/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0287
Epoch 49/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0287
Epoch 50/70
19/19 [==============================] - 0s 3ms/step - loss: 0.0286
Epoch 51/70
19/19 [==============================] - 0s 3ms/step - loss: 0.0286
Epoch 52/70
19/19 [==============================] - 0s 3ms/step - loss: 0.0286
Epoch 53/70
19/19 [==============================] - 0s 3ms/step - loss: 0.0286
Epoch 54/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0286
Epoch 55/70
19/19 [==============================] - 0s 3ms/step - loss: 0.0286
Epoch 56/70
19/19 [==============================] - 0s 2ms/step - loss: 0.0286
Epoch 57/70
19/19 [==============================] - 0s 3ms/step - loss: 0.0286
Epoch 58/70
19/19 [==============================] - 0s 3ms/step - lo

In [ ]:
pd.DataFrame(y_test).to_csv('ypred_tf2.csv')

In [ ]:
pd.DataFrame(y_pred_B).to_csv('ypred_tf2.csv')